<a href="https://colab.research.google.com/github/zahra-zarrabi/HousePrices/blob/master/HousePrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 30.8 MB/s 
     |████████████████████████████████| 170 kB 44.3 MB/s 
     |████████████████████████████████| 133 kB 54.9 MB/s 
     |████████████████████████████████| 97 kB 4.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=bd81e213fcb02a79925c15710ebacb8e6f67a1ec5e00f4284092b85f0d9c7902
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=6b50d4ce9b35995a12e16eb3f9c7ca73f3a1bcffc7e73ed3498abfdc513b3c88
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [ ]:
data_house_train = pd.read_csv('/content/train.csv')
data_house_train = data_house_train.fillna(0)
X_train = data_house_train.iloc[:,1:-1]

Y_train=data_house_train['SalePrice']


In [ ]:
data_house_test = pd.read_csv('/content/test.csv')
data_house_test = data_house_test.fillna(0)
data_price = pd.read_csv('/content/sample_submission.csv')
X_test = data_house_test.iloc[:,1:]


In [ ]:
Data = [X_train,X_test]
data_merge = pd.concat(Data,ignore_index=True)


In [ ]:
for col in data_merge.columns:
  tmp = [isinstance(x, str) for x in data_merge[col].to_list()]
  if any(tmp):
    data_list = np.array(data_merge[col].to_list())
    data_unique = set(data_list)
    cnt = 0
    for x in data_unique:
      data_list[np.where(data_list == x)] = cnt
      cnt += 1
    data_merge[col] = data_list.astype('int')

X_train = data_merge[:X_train.shape[0]]
X_test = data_merge[X_train.shape[0]:]

In [ ]:
cor_sel=np.full((X_train.shape[1]),False,dtype=bool)
X_train
for index, c in enumerate(X_train):
  temp = X_train[c]
  cor = temp.corr(Y_train) 
  if np.abs(cor) > 0.2:
    cor_sel[index]=True
print(cor_sel)
sel_col = X_train.columns[cor_sel]
X_train= X_train[sel_col]


[False False  True  True False False  True False False False False False
 False False False False  True False  True  True  True False  True  True
  True  True  True False False  True False False False  True False False
  True  True False  True  True False  True  True False  True  True False
  True  True False False  True  True False  True  True  True  True  True
  True  True False False  True  True  True False False False False False
 False False False False False  True  True]


In [ ]:
# cor = X_train.corr()
# # print(type(cor))
# cor_sel = np.full((cor.shape[0]),True,dtype=bool)
# for i in range(cor.shape[0]):
#   for j in range(i+1,cor.shape[0]):
#     if cor.iloc[i,j] > 0.7:
#       if cor_sel[j] == True:
#         cor_sel[j]=False
# # print(cor_sel.shape)
# sel_col = X_train.columns[cor_sel]
# X_train= X_train[sel_col]

In [ ]:

X_train=np.array(X_train)
Y_train=np.array(Y_train)



In [ ]:
# data_house_train['BldgType'].unique()

In [ ]:
# x = data_house_train.info()


In [ ]:
X_train_max = np.max(X_train, axis=0)
X_train = X_train / X_train_max

Y_train_max = np.max(Y_train)
Y_train = Y_train / Y_train_max

In [ ]:
X_test = X_test[sel_col]

Y_test=data_price[['SalePrice']]
X_test=np.array(X_test)
Y_test=np.array(Y_test)


X_test_max = np.max(X_test, axis=0)
X_test = X_test / X_test_max

Y_test_max = np.max(Y_test)
Y_test = Y_test / Y_test_max


In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(4,input_dim=X_train.shape[1],activation='relu'),
                                    # tf.keras.layers.Dense(16,activation='relu'),
                                    # tf.keras.layers.Dense(8,activation='relu'),
                                    tf.keras.layers.Dense(1,activation='relu')
])
wandb.init(project="HousePrice")

model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.mse)
model.fit(X_train,Y_train,epochs=50,callbacks=[WandbCallback()],validation_data=(X_test, Y_test))
# model.fit(X_train,Y_train,epochs=40)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 1/50
46/46 [==============================] - 1s 6ms/step - loss: 0.0611 - val_loss: 0.3613
Epoch 2/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0458 - val_loss: 0.3169
Epoch 3/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0363 - val_loss: 0.2927
Epoch 4/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0274 - val_loss: 0.2960
Epoch 5/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0229 - val_loss: 0.2650
Epoch 6/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0197 - val_loss: 0.2736
Epoch 7/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0173 - val_loss: 0.2674
Epoch 8/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0154 - val_loss: 0.2546
Epoch 9/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0136 - val_loss: 0.2532
Epoch 10/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0122 - val_loss: 0.2402
Epoch 11/50
46/46 [

In [ ]:
model.evaluate(X_test,Y_test)

46/46 [==============================] - 0s 979us/step - loss: 0.1849


0.18488900363445282